# Imports

In [ ]:
import os
import csv
import re
import collections
import logging
import itertools
import pickle
import psycopg2
import psycopg2.extras
import dedupe
import numpy as np
import pandas as pd
import simplejson as json

from io import StringIO
from unidecode import unidecode

# Variables

In [ ]:
logging.getLogger().setLevel(logging.INFO)

In [ ]:
configuration_file_name = 'configuration_file_dedupe.json'
config_file_data = {}

In [ ]:
with open(configuration_file_name, 'r') as config_file:
    config_file_data = json.load(config_file)

In [ ]:
# input files
input_file_1 = config_file_data.get('input_file_1')
input_file_2 = config_file_data.get('input_file_2')

# Setup

In [ ]:
def preProcess(column):
    """
    This function does a little bit of data cleaning with the help of Unidecode and Regex libraries.
    Things like casing, extra spaces and new lines can be ignored.
    
    :param column: string object which represents a cell from the csv file
    :return: the preprocessed column
    """
    
    column = unidecode(column)
    column = re.sub('\n', ' ', column)
    column = re.sub('  +', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    
    if not column :
        column = None
        
    return column

In [ ]:
def read_data(filename):
    """
    This function reads CSV file and creates a dictionary of records, 
    where the key is a unique record ID (name of the file + index).
    
    :param filename: string object which represents the name of the
                     input file
    :return: a dictionary object containing all the rows read from the CSV file
    """
    
    data_d = {}
    
    partial_key = filename
    
    with open(filename) as f:
        reader = csv.DictReader(f)
        
        # delete the first 4 characters of the partial key if they are "tmp_"
        if partial_key.find("tmp_") == 0:
            partial_key = partial_key[4:]
            
        for i, row in enumerate(reader):
            clean_row = dict([(k, preProcess(v)) for (k,v) in row.items()])
            data_d[partial_key + str(i)] = dict(clean_row)
            
    return data_d

In [ ]:
def read_dataset_containing_only_the_common_fields_in_both_datasets(input_file, common_fields):
    """
    This function reads from a CSV file only the common fields of both datasets.
    
    :param input_file: string object which represents the name of the
                       input file
    :param common_fields: a list of string objects containing the name of the 
                          common fields of the both datasets
    :return: a dictionary object contai
    """

    # create a temporary file with only common columns
    tmp_input_file_with_common_cols = "tmp_" + input_file

    df = pd.read_csv(input_file, dtype=object)
    df = df[common_fields]

    df.to_csv(tmp_input_file_with_common_cols, index=False)

    # read the data from this temporary file
    data_d = read_data(tmp_input_file_with_common_cols)

    # remove the temporary file 
    os.remove(tmp_input_file_with_common_cols)

    return data_d


In [ ]:
def get_common_fields_of_the_datasets_or_the_given_fields(config_file_data, input_file_1, input_file_2):
    """
    Get the common columns (fields) of both datasets if the user did not specify 
    them in the configuration file.
    
    :param config_file_data: a dictionary containing the information of the
                            configuration file
    :param input_file_1: string object which represents the name of the first dataset
    :param input_file_2: string object which represents the name of the second dataset
    :return: a list containing the common columns of both datasets
    """
    given_fields = []
    
    # if some fields were specified, get them in a list
    if config_file_data['training'].get('field_definitions'):
        field_definitions = config_file_data['training'].get('field_definitions')
        
        for f in field_definitions:
            given_fields.append(f['field'])  
        
    # get the common columns of both datasets
    df1 = pd.read_csv(input_file_1, dtype = object)
    df2 = pd.read_csv(input_file_2, dtype = object)
      
    # postgres only has lower case column names --> make lower case the dataframe column names    
    f1_header_columns = set([x.lower() for x in list(df1.columns.values)])
    f2_header_columns = set([x.lower() for x in list(df2.columns.values)])
    
    common_cols_from_datasets = list(f1_header_columns.intersection(f2_header_columns))
    
    # make sure that the given fields (if given) can be found in the common columns of the datasets
    if len(given_fields) > 0:
        return list(set(given_fields).intersection(set(common_cols_from_datasets)))
    
    return common_cols_from_datasets

In [ ]:
common_columns = get_common_fields_of_the_datasets_or_the_given_fields(config_file_data, input_file_1, input_file_2)

In [ ]:
logging.info('reading records from {}'.format(input_file_1))
first_dataset = read_dataset_containing_only_the_common_fields_in_both_datasets(input_file_1, common_columns)
logging.info('{} records read'.format(len(first_dataset)))

In [ ]:
logging.info('reading records from {}'.format(input_file_2))
second_dataset = read_dataset_containing_only_the_common_fields_in_both_datasets(input_file_2, common_columns)
logging.info('{} records read'.format(len(second_dataset)))

# Training

In [ ]:
logging.info('starting training..')

In [ ]:
def get_training_fields(config_file_data, common_columns):
    """
    Define the common columns (fields) the library will pay attention to 
    by creating a list of dictionaries where each dictionary's keys will be 'field' and
    'type'.
    E.g.: [{'field' : 'field1', 'type' : 'String'}, {'field' : 'field2', 'type' : 'String'}]
    For more information about the accepted types, read more here:
    https://docs.dedupe.io/en/latest//Variable-definition.html
    
    :param config_file_data: dictionary containg the the information of the
                             configuration file
    :param common_columns: a list of string objects containing the name of the 
                           common columns(fields) of the both datasets
    :return: a list containing dictionaries which represents the fields used for training
    """
    if config_file_data['training'].get('field_definitions'):
        fields = []
        for c in common_columns:
            for f in config_file_data['training']['field_definitions']:
                if f['field'] == c:
                    fields.append({'field' : c, 'type' : f['type']})
                    break
                    
        return fields
    
    #if the fields are not specified in configuration file, then create
    #a list of dictionaries where the type of the field will be "String" 
    #by default
    fields = [{'field': f, 'type': 'String'} for f in common_columns]
    
    return fields

In [ ]:
def get_uncertain_pairs(deduper, nr_uncertain_pairs):
    """
    This function gets all the uncertain pairs by using the library's function uncertainPairs.
    
    :param deduper: the library object
    :param nr_uncertain_pairs: how many uncertain pairs the library should give to the user to label
    :return: a list of tuples where every tuple represents a pair of examples which library is uncertain to label them.
    """
    
    uncertain_pairs = [] 
    
    for i in range(0, nr_uncertain_pairs):
        uncertain_pair = deduper.uncertainPairs()
        uncertain_pairs.append(uncertain_pair.pop())
    
    return uncertain_pairs

The first part of the algorithm is the training part. In this part the [RecordLink](https://docs.dedupe.io/en/latest/API-documentation.html#RecordLink) class from Dedupe is used. This class was chosen because the algorithm takes two input files and tries to find a one-to-one match between different examples from the two input data sets.
A RecordLink object is created using the common fields of the two input data sets. If the user specifies these fields in the configuration file, then, the ones that are common to both data sets will be used (they may be just a subset of all the common columns that exist in the input files). These fields are used as features for the classifier. In this part the algorithm will learn different weights for these fields and then it will build a model. Basically, the model is represented by the the fields and their weights.
Regarding the training file, we have three cases:
* In the first case if the training and settings file don't exist, the user should create them. First the algorithm samples some examples to do a 'mini training', that will be used for giving, to the user, the examples that the library is uncertain about. The user should label this examples by typing _y_(yes) in case the user considers that these examples match, _n_(no) in case the examples don't match, _u_(unsure) if the user is unsure about the examples and how to label them, _p_(previous) if the user figures out that he/she didn't label correctly the previous example or _f_(finish) to finish the labeling part. At the end of this process, which is called **active learning**, a training file will be created and the library will use it to do the actual training. The result is a model which will be saved in a settings file. For building the model, the library uses [Regularized Logistic Regression](https://github.com/dedupeio/rlr) classifier.
* In the second case if a training file exists, then the labeling part is skipped and a new model is created/learnt.
* In the third case if a settings file exists, the labeling part and training part is skipped, and the model is read from the settings file.

**Note**: the user can change the classifier that Dedupe uses at this part; more info about this [here](https://docs.dedupe.io/en/latest/API-documentation.html#Dedupe.classifier)

In [ ]:
# if 'create_training_file_by_client' parameter is true this cell will create the 'uncertain_pairs_file'
# configuration parameters used for the training part
nr_examples_sampled_for_training = config_file_data['training'].get('nr_of_examples_for_training')
training_fields = get_training_fields(config_file_data, common_columns)
create_training_file = config_file_data['training']['create_training_file_by_client']
training_file = config_file_data['training'].get('training_file')
settings_file = config_file_data['training'].get('settings_file')

if settings_file:
    logging.info('reading from {}'.format(settings_file))
    with open(settings_file, 'rb') as sf :
        linker = dedupe.StaticRecordLink(sf)
        
else:
    linker = dedupe.RecordLink(training_fields)

    linker.sample(first_dataset, second_dataset, nr_examples_sampled_for_training)
     
    # if the user wants to create a training file on the client side we will make available
    # through a GET request a binary file containing 200 pairs of examples 
    # that Dedupe doesn't know if they match or not; then, on the client side
    # the user will label the pairs (match or distinct) and the client will
    # make a POST request with the newly created training file
    if create_training_file:
        uncertain_pairs = get_uncertain_pairs(linker, 200)
        with open("uncertain_pairs_file", "wb") as f:
            pickle.dump(uncertain_pairs, f)

    else:     
        if training_file:
            logging.info('reading labeled examples from {}'.format(training_file))
            with open(training_file) as tf :
                linker.readTraining(tf)
        else:    
            logging.info('starting active labeling...')
            dedupe.consoleLabel(linker)

        linker.train()

        # if the training and settings files were not specified, but you want to keep them between runs, 
        # rename them or save them somewhere else, because they will be overwritten every time the algorithm is run
        if not training_file:
            with open("training_file.json", 'w') as tf:
                linker.writeTraining(tf)

        if not settings_file:
            with open("settings_file", 'wb') as sf:
                linker.writeSettings(sf)

        linker.cleanupTraining()

In [ ]:
if settings_file is None and create_training_file:
    logging.info('reading labeled examples from the newly created training file {}'.format(training_file))
    
    with open(training_file) as tf:
        linker.readTraining(tf)

    linker.train()
    
    with open("settings_file", 'wb') as sf:
        linker.writeSettings(sf)

    linker.cleanupTraining()

# Clustering

This part of the algorithm consists of creating clusters. Before, a threshold value should be calculated using the [threshold](https://docs.dedupe.io/en/latest/API-documentation.html#RecordLink.threshold) method from Dedupe library. Threshold is a float number between 0 and 1. The records will be considered as potential duplicates if the predicted probability of being duplicates is above this threshold. Lowering the threshold number will increase recall, raising it will increase precision. For reducing the time of computing the threshold, a given number of examples are sampled from both datasets. 

Based on the parameters given in the configuration fie, we have two cases:
* The threshold value is given by the user and then the computing part is skipped.
* In case the threshold value is not given, the user should specify, in the configuration file, the _recall_weight_ parameter and the _nr_of_sample_data_for_threshold_ parameter. _Recall_ weight is used for computing the threshold and _nr_of_sample_data_for_threshold_ is used for getting the samples of examples from the input data sets.

In [ ]:
# parameters used for the 'threshold' method from Dedupe
threshold_value = None

if config_file_data.get('threshold') or config_file_data.get('compute_threshold'):
    if config_file_data.get('threshold'):
        threshold_value = config_file_data.get('threshold')
    else:
        logging.info('find the best threshold...')

        recall_weight = config_file_data['compute_threshold'].get('recall_weight')

        sample_nr_of_examples_for_threshold = \
            config_file_data['compute_threshold'].get('nr_of_sample_data_for_threshold')

        # get n examples from the input dataset, where n = 'sample_nr_of_examples_for_threshold' 
        sample_data_threshold_first_dataset = {
            k: first_dataset[k] for k in list(first_dataset)[:int(sample_nr_of_examples_for_threshold)]
        }
        sample_data_threshold_second_dataset = {
            k: second_dataset[k] for k in list(second_dataset)[:int(sample_nr_of_examples_for_threshold)]
        }

        threshold_value = linker.threshold(sample_data_threshold_first_dataset,
                                           sample_data_threshold_second_dataset,
                                           recall_weight)


For the clustering part the [match](https://docs.dedupe.io/en/latest/API-documentation.html#RecordLink.match) method from Dedupe library is used. This method returns a list which contains tuples of record ids, of the input examples that match, and also their scores.

In [ ]:
logging.info('clustering...')
print(threshold_value)

if threshold_value:
    linked_records = linker.match(first_dataset, second_dataset, threshold_value)
else:
    linked_records = linker.match(first_dataset, second_dataset)

logging.info('# duplicate sets {}'.format(len(linked_records)))

In [ ]:
# the maximum value for cluster_id found in the database
# we will create new clusters, which will have ids starting from this value onward
cluster_id = int(config_file_data.get('last_cluster_id')) if config_file_data.get('last_cluster_id') else 0

In [ ]:
# This cell creates a dictionary where the keys will be the record id given
# in the read_data method and the values will be tuples which
# contain the cluster id and the score. The output files are created using this
# dictionary

cluster_membership = {}

for cluster, score in linked_records:
    cluster_id += 1    
    for record_id in cluster:
        cluster_membership[record_id] = (cluster_id, score) 

unique_id = cluster_id + 1

# Output

This part creates two output files based on the input files, but adding two more columns: _cluster_id_ and _link_score_.

**Cluster_id** column represents a number which is assigned to two examples that match. A _cluster_id_ will also be assigned to one example from one dataset, in case it doesn't have any matches in the other dataset.

**Link_score** column represents a measurement of how similar two examples from the same cluster are.

In [ ]:
def create_output_file(filename, output_file):
    """
    Create an output file which contains cluster id and the link score columns
    besides the initial columns from the input file 
     
    :param filename: string object which represents the name of one dataset
    :param output_file: string object which represents the name of the output
                       file
    """

    global unique_id

    with open(output_file, 'w') as f:
        writer = csv.writer(f)
        with open(filename) as f_input:
            reader = csv.reader(f_input)

            heading_row = next(reader)
            heading_row.insert(0, 'link_score')
            heading_row.insert(0, 'cluster_id')
            writer.writerow(heading_row)

            for row_id, row in enumerate(reader):
                cluster_details = cluster_membership.get(filename + str(row_id))

                # the examples which have not a match with other examples will be put
                # in their own cluster
                if cluster_details is None:
                    cluster_id = unique_id
                    unique_id += 1
                    score = None
                else:
                    cluster_id, score = cluster_details

                row.insert(0, score)
                row.insert(0, cluster_id)
                writer.writerow(row)


In [ ]:
output_file_1 = "output_" + input_file_1
output_file_2 = "output_" + input_file_2

In [ ]:
logging.info('create output files...')
create_output_file(input_file_1, output_file_1)
create_output_file(input_file_2, output_file_2)

# Evaluation

In this part the clusters made by library are evaluated. We want to see how good the algorithm performed. For evaluation, [precision](https://en.wikipedia.org/wiki/Precision_and_recall) and [recall](https://en.wikipedia.org/wiki/Precision_and_recall) are being used.

In [ ]:
logging.info('starting evaluation...')

In [ ]:
def get_merged_dataframe_containing_only_cluster_id_and_label_column(output_file_1, output_file_2, columns):
    """
    Create a merged dataframe which contains only the cluster id and label("ground truth") column.
    
    :param output_file_1: string object which represents the name of the first output file
    :param output_file_2: string object which represents the name of the second output file
    :param columns: a list of string objects containg the name of the common columns
    :return: a dataframe
    """
    df1 = pd.read_csv(output_file_1)
    df2 = pd.read_csv(output_file_2)

    df1 = df1[columns]
    df2 = df2[columns]

    frames = [df1, df2]
    
    return pd.concat(frames, axis = 0)

In [ ]:
def sql_statement_for_creating_new_table(column_datatypes, tmp_table_name):
    """
    Create a general statement for creating a table in a database.

    :param column_datatypes: list of string objects, where the strings represents the datatypes
                             of each column
    :param tmp_table_name: string containing the name of the new table
    :return: a string object which represents the SQL statement.
    """
    create_table_sql_statement = "CREATE TABLE " + tmp_table_name + " (company_id SERIAL PRIMARY KEY"

    for k, v in column_datatypes.items():
        create_table_sql_statement += ","
        create_table_sql_statement += k
        create_table_sql_statement += " "
        create_table_sql_statement += v

    return create_table_sql_statement + ")"

In [ ]:
def sql_statement_for_copying_values_from_file(columns, tmp_table_name):
    """
    Create a general statement for copying values from a csv file in a specific table.

    :param columns: list of string objects, where the strings are names of columns
    :param tmp_table_name: string containing the name of the table where values will
                         be inserted (copied) 
    :return: a string object which represents the SQL statement
    """
    
    sql_copy_statement = "COPY " + tmp_table_name + " ("
    
    for c in columns:
        sql_copy_statement += c + ", "
    
    return sql_copy_statement[:len(sql_copy_statement) - 2] + ") FROM STDIN CSV HEADER"

In [ ]:
def get_columns_and_their_datatypes(result_df):
    """
    Convert the dataframe's datatypes into postgres datatypes.
    
    :param result_df: pandas dataframe. 
    :return: a dictionary object where the keys are the columns and the values represents their datatypes.
    """
    
    column_datatypes = {}

    for k, v in dict(result_df.dtypes).items():
        if v == 'int64':
            column_datatypes[k] = 'INT'
        elif v == 'float64':
            column_datatypes[k] = 'FLOAT'
        else:
            column_datatypes[k] = 'VARCHAR(500)'
    
    return column_datatypes

In [ ]:
def evaluateDuplicates(found_dupes, true_dupes):
    """
    Calculate precision and recall.
    
    :param found_dupes: result from 'dupePairs' function (the result after we apply the algorithm)
    :param true_dupes: result from 'dupePairs' function (the ground truth)
     
    """

    true_positives = found_dupes.intersection(true_dupes)
    false_positives = found_dupes.difference(true_dupes)

    if len(found_dupes) == 0:
        precision = None
    else:
        precision = 1 - len(false_positives) / float(len(found_dupes))

    if len(true_dupes) == 0:
        recall = None
    else:
        recall = len(true_positives) / float(len(true_dupes))

    logging.info('precision {}'.format(precision))

    logging.info('recall {}'.format(recall))


In [ ]:
def dupePairs(colname_id, colname_index, table_name):
    """
    This function creates a set which contains pairs (tuples) of numbers.
    The pair is an ordered tuple containing two company ids. The tuple contains records that were put in
    the same cluster.
    
    A company id is the primary key of a table, where we have all the companies, thus, this company id
    is unique in that table.
    
    When we do the evaluation, we extract the company ids by querying the cluster_id column to find the
    clusters that Dedupe made, and, with another call to this function we extract the company ids by
    querying the "label" column of the companies to get the true clusters (the ground truth). Then,
    the 'evaluateDuplicates' function receives these two sets of tuples and compares them to calculate
    precision and recall
 
    :param colname_id: string object containing the name of the column which represents the label
                       of the examples (e.g cluster id or id) 
    :param colname_index: string object which represents the record id (the PK from the table)
    :param table_name: string object which represents the name of the table from where we extract
                       the examples 
    :return: a set which contains pairs (tuples) of number
    """
    
    colname_id = colname_id.split()[0]
    colname_index = colname_index.split()[0]
    table_name = table_name.split()[0]

    dupe_d = collections.defaultdict(list)

    db_cursor.execute("SELECT " + colname_id + " FROM " + table_name)
    rows_id = np.array(db_cursor.fetchall())

    db_cursor.execute("SELECT " + colname_index + " FROM " + table_name)
    rows_company = np.array(db_cursor.fetchall())

    for row_id, company_id in zip(rows_id, rows_company):
        dupe_d[row_id[colname_id]].append(company_id[colname_index])

    dupe_s = set([])
    for (unique_id, cluster) in dupe_d.items():
        if len(cluster) > 1:
            for pair in itertools.combinations(cluster, 2):
                dupe_s.add(frozenset(pair))

    return dupe_s


In [ ]:
if config_file_data.get('evaluation'):
    # evaluation parameters
    # precision and recall is done with a database
    db_connection = psycopg2.connect(
        database=config_file_data['database_config'].get('database_name'),
        user=config_file_data['database_config'].get('username'),
        password=config_file_data['database_config'].get('password'),
        host=config_file_data['database_config'].get('host'),
        port=config_file_data['database_config'].get('port'),
        cursor_factory=psycopg2.extras.RealDictCursor
    )
    db_cursor = db_connection.cursor()

    # this is the name of the column based on which we can create correct clusters (the label)
    # for example if the column is a unique id, then we can see which companies match based on it
    # N.B.: This only makes sense if you will NOT use this column in the training process, i.e.,
    # don't give it as a training field!
    label_column_name = config_file_data['evaluation'].get('label_column_name')
    tmp_table_name = "tmp_test_table"
    columns = ['cluster_id', label_column_name]

    result_df = get_merged_dataframe_containing_only_cluster_id_and_label_column(output_file_1, output_file_2, columns)

    # write the dataframe as a csv looking file into a string and explicitly make the file pointer point at 
    # the beginning of the file
    s_buf = StringIO()
    result_df.to_csv(s_buf, index=False)
    s_buf.seek(0)

    # convert the dataframe datatypes into postgres datatypes
    column_datatypes = get_columns_and_their_datatypes(result_df)

    logging.info('importing raw data from csv...')
    db_cursor.execute("DROP TABLE IF EXISTS " + tmp_table_name)

    db_cursor.execute(sql_statement_for_creating_new_table(column_datatypes, tmp_table_name))
    db_connection.commit()

    db_cursor.copy_expert(sql_statement_for_copying_values_from_file(columns, tmp_table_name), s_buf)
    db_connection.commit()

    logging.info('generating the true and test clusters...')
    true_dupes = dupePairs(label_column_name, 'company_id', tmp_table_name)
    test_dupes = dupePairs('cluster_id', 'company_id', tmp_table_name)
    
    # True dupes represents a list containing the real clusters which are made using the label column.(for example,
    # the id column)
    # Found dupes represents a list containing the clusters found by the library
    logging.info("True dupes: {}".format(len(true_dupes)))
    logging.info("Found dupes: {}".format(len(test_dupes)))
    evaluateDuplicates(test_dupes, true_dupes)

    db_cursor.close()
    db_connection.close()
